<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/cohort_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Preparing IDC Cohorts for Image Processing**

This notebook is one of the examples that accompany NCI Imaging Data Commons. IDC example notebooks are located in this repository: https://github.com/ImagingDataCommons/IDC-Examples/tree/master/notebooks.

In this example we show how to prepare a cohort for image processing, by converting the DICOM images into NRRD or NIfTI files. We also show how to visualise such data in a notebook, and how to convert the processing results, stored in one of the aforementioned formats, into DICOM SEG for more advanced purposes (with softwares like 3D slicer).

To learn more about the [cohort definition through the IDC Portal](https://portal.imaging.datacommons.cancer.gov/) and the [cohort download](https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/Cohort_download.ipynb), you can refer to the [IDC User Guide](https://learn.canceridc.dev/) and the example notebooks provided with the [IDC-Examples repository](https://github.com/ImagingDataCommons/IDC-Examples).

---

## **Init & Imports**

In order to access the IDC resources, the user must complete at first the Google authentication process.

To continue with the notebook, when prompted by the next code cell follow the generated link. After granting Google Cloud SDK access to the selected Google account, you will get a one-use login code. Copy the code, paste it in the blank space under the cell, and press enter to complete the authentication procedure.

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# useful information

curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print("Current directory :", curr_dir[-1])
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])

Current directory : /content
Hostname          : e3f10b17fa82
Username          : root


To proceed with the cells below you will also need to:
* Initialize `myProjectID` in the cell below with a project ID that you can bill
* Generate a table name `cohortBQTable` through IDC (be aware you will also need to paste this manually in the BigQuery download cell)

In [ ]:
## INSERT YOUR OWN PROJECT-ID AND BIGQUERY TABLE NAME HERE!
cohortBQTable = "##COHORT_BQ_TABLE##"
myProjectID = "##MY_PROJECT_ID##"

Before continuing with the notebook, we need to set-up the Google Colab environment by installing the Python and system dependencies the pre-processing and processing pipeline rely on.

### **Python Dependencies**

Install and import all the Python dependencies.

The main python packages we need to install are:

* [pydicom](https://github.com/pydicom/pydicom), a Python package that lets the use read, modify, and write DICOM data in an easy "pythonic" way;
* [itkwidgets](https://github.com/InsightSoftwareConsortium/itkwidgets), a Python package for interactive visualization of images, point sets, and meshes in notebook environments;
* [SimpleITK](https://simpleitk.readthedocs.io/en/master/gettingStarted.html), a Python package which provides a simplified interface to ITK.



In [ ]:
%%capture
!pip install pyplastimatch
!pip install ipywidgets

In [ ]:
import os
import sys
import shutil

import json
import pprint
import numpy as np
import pandas as pd

import pydicom
import nibabel as nib
import SimpleITK as sitk

print("Python version               : ", sys.version.split('\n')[0])
print("Numpy version                : ", np.__version__)

# ----------------------------------------

#everything that has to do with plotting goes here below
import matplotlib
matplotlib.use("agg")

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

%matplotlib inline
%config InlineBackend.figure_format = "png"

import ipywidgets as ipyw

## ----------------------------------------

# create new colormap appending the alpha channel to the selected one
# (so that we don't get a \"color overlay\" when plotting the segmask superimposed to the CT)
cmap = plt.cm.Reds
my_reds = cmap(np.arange(cmap.N))
my_reds[:, -1] = np.linspace(0, 1, cmap.N)
my_reds = ListedColormap(my_reds)

cmap = plt.cm.Greens
my_greens = cmap(np.arange(cmap.N))
my_greens[:, -1] = np.linspace(0, 1, cmap.N)
my_greens = ListedColormap(my_greens)

cmap = plt.cm.Blues
my_blues = cmap(np.arange(cmap.N))
my_blues[:, -1] = np.linspace(0, 1, cmap.N)
my_blues = ListedColormap(my_blues)

cmap = plt.cm.spring
my_spring = cmap(np.arange(cmap.N))
my_spring[:, -1] = np.linspace(0, 1, cmap.N)
my_spring = ListedColormap(my_spring)
## ----------------------------------------

import seaborn as sns

Python version               :  3.7.13 (default, Apr 24 2022, 01:04:09) 
Numpy version                :  1.21.6


### **System Dependencies**

Install all the system dependencies. 

The only package we will need for this tutorial is [Plastimatch](https://plastimatch.org/index.html). Plastimatch is considered to be the swiss army knife of medical images processing: we will use it to convert DICOM (CT, RTSTRUCT) series to NRRD files - but it can be used for a multitude of other tasks, such as registration, resampling, cropping, and computing statistics to name a few. Plastimatch is also available as a 3DSlicer plug-in and can be used directly from the Slicer GUI.

For the sake of clarity and simplicity, we will call Plastimatch from a very simple [Python wrapper](https://github.com/AIM-Harvard/pyplastimatch) written for the occasion (unfortunately, Plastimatch does not provide an official one) - more on this later.

In [ ]:
%%capture
!sudo apt update

!sudo apt install plastimatch

In [ ]:
!echo $(plastimatch --version)

plastimatch version 1.7.0


We are also going to install subversion, a tool that will allow us to clone GitHub repositories only partially (to save time and space).

In [ ]:
%%capture

!sudo apt install subversion

In [ ]:
!echo $(svn --version | head -n 2)

svn, version 1.9.7 (r1800392) compiled Mar 28 2018, 08:49:13 on x86_64-pc-linux-gnu


---

## **Cohort Download**

As the Imaging Data Commons GCS buckets are "[requester pays](https://cloud.google.com/storage/docs/requester-pays)" buckets, it is not possible to [mount such buckets directly in Colab](https://gist.github.com/korakot/f3600576720206363c734eca5f302e38).

Instead, what the users can do is to exploit the BigQuery table/manifest files associated with the dataset(s), select the cohort(s) of interest, and then download the files exploiting `gsutil`. BigQuery export of the cohort manifest is the recommended approach (see the [Cohort Download Notebook](https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/Cohort_download.ipynb) for a more detailed explanation).

When exporting the manifest as a file, cohorts larger than 65,000 items will be exported as multiple files, and only up to 10 files for a multi-part cohort manifest can be exported. BigQuery manifest export does not have any limitations, and can be used in the same manner no matter how large or small is the cohort you want to export.

**Note**: the cohort selection operation can be done through the graphic user interface at the [IDC Portal](https://portal.imaging.datacommons.cancer.gov/). If you want to learn more, please refer to the [IDC User Guide](https://learn.canceridc.dev/). A simple example is provided at the "Cohort Download" section of [this notebook tutorial](https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/lung_nodules_demo.ipynb#scrollTo=WwydG1e170xU).

### **Downloading the Data from GCS**

In the next cell, make sure to fill in the name of your GCP project and paste the query we obtained following the cells above.

At the end, it should look like the following:

```
%%bigquery --project=my-gcp-project cohort_df

SELECT * FROM `canceridc-user-data.user_manifests.manifest_cohort_222_20210811_071925`
```

In this tutorial, we are going to run the simple data preparation pipeline on CT scans of patients from the [Non-Small Cell Lung Cancer Radiomics (NSCLC-Radiomics)](https://wiki.cancerimagingarchive.net/display/Public/NSCLC-Radiomics) dataset.

<br>

If you are interested in processing cohorts with multiple modalities, or other types of data (e.g., RTDOSE data), you can still do so using the functions we provide in the Plastimatch wrapper, writing your own, or invoking Plastimatch from command line in this notebook using the `!` magic. If you seek advice for your specific use case or encounter difficulties (or bugs!) in the data preparation process, you are welcome to get in touch with us at the [IDC discussion forum](https://discourse.canceridc.dev/) - we will be happy to help!

In [ ]:
## INSERT YOUR OWN PROJECT-ID AND BIGQUERY TABLE NAME HERE!
%%bigquery --project=$myProjectID cohort_df

SELECT * 
FROM `##COHORT_BQ_TABLE##`

After populating the Pandas DataFrame with the result of the bigquery query, we can inspect it to make sure it looks ok.

As you will see, each DICOM slice has its own entry (that's why one has to remove `LIMIT 1000` from the query to get the complete result) and a series of useful information:
* on the dataset it belongs to - especially useful in the case multiple datasets are exported as part of an IDC cohort - such as the `collection_id` and the `source_DOI`;
* on the patient, study, and series it belongs to, such as the `PatientID`, `StudyInstanceUID` and `SeriesInstanceUID`;
* on the GCS bucket location and organisation, such as `gcs_url`, `crdc_instance_uuid`, and `crdc_series_uuid`.



In [ ]:
cohort_df.head()

,PatientID,collection_id,source_DOI,StudyInstanceUID,SeriesInstanceUID,SOPInstanceUID,crdc_study_uuid,crdc_series_uuid,crdc_instance_uuid,gcs_url,idc_version,access
0,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.1885589837633001625176...,d5cb521c-8761-4e6d-a371-306fab5c956e,6e0f8b4e-a116-477d-822b-5adc13b764ae,02904ce9-89a7-41a0-b4af-3b8d2ecdb5a7,gs://idc-open-cr/02904ce9-89a7-41a0-b4af-3b8d2...,10.0,Public
1,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.2176279587798300737446...,d5cb521c-8761-4e6d-a371-306fab5c956e,6e0f8b4e-a116-477d-822b-5adc13b764ae,029aa22a-b3d2-4dc7-95c8-822c8e2a1449,gs://idc-open-cr/029aa22a-b3d2-4dc7-95c8-822c8...,10.0,Public
2,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.5520085834678520978071...,d5cb521c-8761-4e6d-a371-306fab5c956e,6e0f8b4e-a116-477d-822b-5adc13b764ae,07610515-ec98-4852-bdd8-0dd5a91f5478,gs://idc-open-cr/07610515-ec98-4852-bdd8-0dd5a...,10.0,Public
3,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.2048931361272395047520...,d5cb521c-8761-4e6d-a371-306fab5c956e,6e0f8b4e-a116-477d-822b-5adc13b764ae,07cade33-bdd5-4ee9-bac0-b0223945aa7c,gs://idc-open-cr/07cade33-bdd5-4ee9-bac0-b0223...,10.0,Public
4,LUNG1-002,nsclc_radiomics,10.7937/K9/TCIA.2015.PF0M9REI,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.32722.99.99.7807413591260762594939...,d5cb521c-8761-4e6d-a371-306fab5c956e,6e0f8b4e-a116-477d-822b-5adc13b764ae,0c795f59-05ce-4b9a-b8c8-56a46eba2910,gs://idc-open-cr/0c795f59-05ce-4b9a-b8c8-56a46...,10.0,Public


In [ ]:
# create the directory tree
!mkdir -p tutorial 
!mkdir -p tutorial/data tutorial/output
!mkdir -p tutorial/data/tmp tutorial/data/dicom tutorial/data/processed

In [ ]:
gs_file_path = "tutorial/data/gcs_paths.txt"

# to speed up the process, download only a portion of the dataset
pat_list = np.unique(cohort_df["PatientID"].values)
n_pat = len(pat_list)

random_idxs = np.random.choice(n_pat, size = 5, replace = False)

sliced_df = cohort_df[cohort_df["PatientID"].isin(list(pat_list[random_idxs]))]
sliced_df["gcs_url"].to_csv(gs_file_path, header = False, index = False)

In [ ]:
%%capture
# https://cloud.google.com/storage/docs/gsutil/commands/cp
!mkdir -p tutorial/data/tmp
!cat $gs_file_path | gsutil -u $myProjectID -m cp -Ir tutorial/data/tmp

As you will notice, for instance, by browsing the `tutorial/data/tmp` folder, the DICOM files will not be organised by default following patients, series, or studies.

To organise the data in a more common (and human-understandable) fashion, we will make use of [DICOMSort](https://github.com/pieper/dicomsort). DICOMSort is an open source tool for custom sorting and renaming of dicom files based on their specific DICOM tags. In our case, we will exploit DICOMSort to organise the DICOM data by `PatientID`, `StudyInstanceUID`, and `SeriesInstanceUID` - so that the final directory will look like the following:

```
tutorial/data/dicom/$PatientID
 └─── $StudyInstanceUID
       ├─── $SeriesInstanceUID_CT
       │     ├─── $SOPInstanceUID_slice0.dcm
       │     ├─── $SOPInstanceUID_slice1.dcm
       │     ├───  ...
       │
       ├─── $SeriesInstanceUID_RTSTRUCT
       │     └─── $SOPInstanceUID_RTSTRUCT.dcm
       │
       └─── $SeriesInstanceUID_RTSEG
             └─── $SOPInstanceUID_RTSEG.dcm

```

Furthermore, to prepare the data for processing (e.g., data conversion from DICOM) we will make use of [PyPlastimatch](https://github.com/denbonte/pyplastimatch/), a very basic python wrapper for Plastimatch, written for the purpose of this tutorial. The wrapper also contains a very basic yet interactive visualisation function we will be using in the last part of the notebook to check the segmentation results.

In [ ]:
# DICOMSORT - for re-organising DICOM files in folders
!git clone https://github.com/pieper/dicomsort.git tutorial/dicomsort

Cloning into 'tutorial/dicomsort'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 130 (delta 0), reused 1 (delta 0), pack-reused 126
Receiving objects: 100% (130/130), 44.12 KiB | 564.00 KiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
# run the DICOMSort command
!python tutorial/dicomsort/dicomsort.py -u tutorial/data/tmp tutorial/data/dicom/%PatientID/%Modality/%SOPInstanceUID.dcm

# get rid of the temporary folder, storing the unsorted DICOM data 
!rm -r tutorial/data/tmp

100% 599/599 [00:07<00:00, 83.69it/s] 
Files sorted


---

## **Data Pre-processing**


Using the simple Plastimatch wrapper, let's convert the DICOM CT series in both NRRD and NIfTI format. Furthermore, we are going to convert the DICOM RTSTRUCT Series into a NRRD volume as well.


In [ ]:
import pyplastimatch as pypla

from pyplastimatch.utils import viz as viz_utils

### **DICOM to NRRD/NIfTI Conversion**

Let's use the functions from the simple wrapper to convert the data into the formats mentioned above exploiting Plastimatch.

In [ ]:
from IPython.display import clear_output

pat_list = list(np.unique(sliced_df["PatientID"].values))

for pat in pat_list:

  # clear cell output before processing patient data 
  clear_output(wait=True)

  # directories storing NRRD and NIfTI files
  base_preproc_path = "tutorial/data/processed"

  pat_dir_path_nrrd = os.path.join(base_preproc_path, "nrrd", pat)
  pat_dir_path_nii = os.path.join(base_preproc_path, "nii", pat)
    
  # patient subfolder where all the preprocessed NRRDs will be stored
  if not os.path.exists(pat_dir_path_nrrd): os.makedirs(pat_dir_path_nrrd)
    
  # patient subfolder where all the preprocessed NIfTIs will be stored
  if not os.path.exists(pat_dir_path_nii): os.makedirs(pat_dir_path_nii)

  # path to the directory where the DICOM CT file is stored
  path_to_ct_dir = os.path.join("tutorial/data/dicom", pat, "CT")

  # path to the files where the NRRD and NIfTI CTs will be stored
  ct_nrrd_path = os.path.join(pat_dir_path_nrrd, pat + "_ct.nrrd")
  ct_nii_path = os.path.join(pat_dir_path_nii, pat + "_ct.nii.gz")


  # path to the directory where the DICOM RTSTRUCT file is stored
  path_to_rt_dir = os.path.join("tutorial/data/dicom", pat, "RTSTRUCT")

  # path to the files where the NRRD RTSTRUCTs will be stored
  rt_folder = os.path.join(pat_dir_path_nrrd, "RTSTRUCT")
  rt_list_path = os.path.join(pat_dir_path_nrrd, "RTSTRUCT_content")

  verbose = True

  # logfile for the plastimatch conversion
  log_file_path_nrrd = os.path.join(pat_dir_path_nrrd, pat + '_pypla.log')
  log_file_path_nii = os.path.join(pat_dir_path_nii, pat + '_pypla.log')
    
  # DICOM CT to NRRD conversion (if the file doesn't exist yet)
  if not os.path.exists(ct_nrrd_path):
    convert_args_ct = {"input" : path_to_ct_dir,
                      "output-img" : ct_nrrd_path}
    
    # clean old log file if it exist
    if os.path.exists(log_file_path_nrrd): os.remove(log_file_path_nrrd)
    
    pypla.convert(verbose = verbose, path_to_log_file = log_file_path_nrrd, **convert_args_ct)

  # DICOM RTSTRUCT to NRRD conversion (if the file doesn't exist yet)
  if not os.path.exists(rt_folder):
    convert_args_rt = {"input" : path_to_rt_dir, 
                      "referenced-ct" : path_to_ct_dir,
                      "output-prefix" : rt_folder,
                      "prefix-format" : 'nrrd',
                      "output-ss-list" : rt_list_path}
    
    # clean old log file if it exist
    if os.path.exists(log_file_path_nrrd): os.remove(log_file_path_nrrd)
    
    pypla.convert(verbose = verbose, path_to_log_file = log_file_path_nrrd, **convert_args_rt)

  # DICOM CT to NIfTI conversion (if the file doesn't exist yet)
  if not os.path.exists(ct_nii_path):
    convert_args_nii = {"input" : path_to_ct_dir, 
                        "output-img" : ct_nii_path}
    
    # clean old log file if it exist
    if os.path.exists(log_file_path_nii): os.remove(log_file_path_nii)
    
    pypla.convert(verbose = verbose, path_to_log_file = log_file_path_nii, **convert_args_nii)


Running 'plastimatch convert' with the specified arguments:
  --input tutorial/data/dicom/LUNG1-422/CT
  --output-img tutorial/data/processed/nrrd/LUNG1-422/LUNG1-422_ct.nrrd
... Done.

Running 'plastimatch convert' with the specified arguments:
  --input tutorial/data/dicom/LUNG1-422/RTSTRUCT
  --referenced-ct tutorial/data/dicom/LUNG1-422/CT
  --output-prefix tutorial/data/processed/nrrd/LUNG1-422/RTSTRUCT
  --prefix-format nrrd
  --output-ss-list tutorial/data/processed/nrrd/LUNG1-422/RTSTRUCT_content
... Done.

Running 'plastimatch convert' with the specified arguments:
  --input tutorial/data/dicom/LUNG1-422/CT
  --output-img tutorial/data/processed/nii/LUNG1-422/LUNG1-422_ct.nii.gz
... Done.


The folder `/content/tutorial/data/processed` now stores both NIfTI and NRRD files (storing both the CT and the RTSTRUCT data) ready to be ingested by your image processing pipeline.

For further pre-processing operations, you can check out what we implemented in our basic [Plastimatch wrapper](https://github.com/AIM-Harvard/pyplastimatch) (a basic example of data resampling is provided [at this notebook](https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/lung_nodules_demo.ipynb#scrollTo=yWPkmQKuBGOO)).

---

## **Data Visualisation**

Simple interactive axial slice (and segmentation) visualisation.

In [ ]:
random_pat = pat_list[np.random.randint(len(pat_list))]

pat_dir_path_nrrd = os.path.join("/content/tutorial/data/processed/nrrd/", random_pat)

assert os.path.exists(pat_dir_path_nrrd)

ct_nrrd_path = os.path.join(pat_dir_path_nrrd, "%s_ct.nrrd"%(random_pat))
ct_nrrd = sitk.GetArrayFromImage(sitk.ReadImage(ct_nrrd_path))

rt_folder_path = os.path.join(pat_dir_path_nrrd, "RTSTRUCT")
rt_list = os.listdir(rt_folder_path)

segmask_dict = dict()
segmask_cmap_dict = dict()

cmap_list = [my_spring, my_reds, my_blues, my_greens]

for rt_vol_name in rt_list:

  structure_name = rt_vol_name.split(".nrrd")[0]

  rt_vol_nrrd_path = os.path.join(pat_dir_path_nrrd, "RTSTRUCT", rt_vol_name)

  segmask_dict[structure_name] = sitk.GetArrayFromImage(sitk.ReadImage(rt_vol_nrrd_path))
  segmask_cmap_dict[structure_name] = cmap_list[np.random.randint(len(cmap_list))]

In [ ]:
_ = viz_utils.AxialSliceSegmaskViz(ct_volume = ct_nrrd,
                         segmask_dict = segmask_dict,
                         segmask_cmap_dict = segmask_cmap_dict,
                         dpi = 100, figsize = (8, 8))

interactive(children=(Output(),), _dom_classes=('widget-interact',))